In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import joblib
import numpy as np
import pandas as pd
from models import PM10_CNN_LSTM, PM25_CNN_LSTM, carbon_monoxide_CNN_LSTM, nitrogen_dioxide_CNN_LSTM, ozone_CNN_LSTM, sulphur_dioxide_CNN_LSTM, dust_CNN_LSTM

## A function for model finetuning

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and scaler function
def load_model_and_scaler(pollutant):
    model = torch.load(f'{pollutant}_model.pt')
    scaler = joblib.load(f'{pollutant}_scaler.pkl')
    data = pd.read_csv(f'update_{pollutant}.csv', parse_dates=['date'], index_col='date')
    return model, scaler, data

# Create sequences function
def create_sequences(data, lookback):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

# Fine-tuning function
# Fine-tune pollutant model function
def fine_tune_pollutant_model(pollutant, lookback=300, epochs=2):

    # display the pollutant 
    print(f'Fine-tuning {pollutant} model')
    # Load model, scaler, and new data
    model, scaler, data = load_model_and_scaler(pollutant)

    new_scaled_data = scaler.transform(data)

    # Create sequences from new data
    X_new, y_new = create_sequences(new_scaled_data, lookback)
    X_new, y_new = torch.tensor(X_new, dtype=torch.float32), torch.tensor(y_new, dtype=torch.float32)

    # Move to device
    X_new, y_new = X_new.to(device), y_new.to(device)
    model = model.to(device)

    # Define loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Fine-tune the model
    model.train()
    for epoch in range(epochs):
        for i in range(len(X_new)):
            inputs = X_new[i:i+1]
            targets = y_new[i:i+1]

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')


    # Save the fine-tuned model
    torch.save(model, f'{pollutant}_model.pt')

# Fine-tune all pollutants function
def fine_tune_all_pollutants(lookback=300, epochs=3):
    pollutants = ['carbon_monoxide', 'dust', 'nitrogen_dioxide', 'ozone', 'pm_10', 'pm_25', 'sulphur_dioxide']
    
    for pollutant in pollutants:
        fine_tune_pollutant_model(pollutant, lookback, epochs)



In [3]:
fine_tune_all_pollutants(lookback=300, epochs=5)

Fine-tuning carbon_monoxide model
Epoch [1/5], Loss: 0.0017
Epoch [2/5], Loss: 0.0011
Epoch [3/5], Loss: 0.0009
Epoch [4/5], Loss: 0.0006
Epoch [5/5], Loss: 0.0005
Fine-tuning dust model
Epoch [1/5], Loss: 0.0042
Epoch [2/5], Loss: 0.0037
Epoch [3/5], Loss: 0.0035
Epoch [4/5], Loss: 0.0034
Epoch [5/5], Loss: 0.0028
Fine-tuning nitrogen_dioxide model
Epoch [1/5], Loss: 0.0366
Epoch [2/5], Loss: 0.0242
Epoch [3/5], Loss: 0.0174
Epoch [4/5], Loss: 0.0123
Epoch [5/5], Loss: 0.0090
Fine-tuning ozone model
Epoch [1/5], Loss: 0.0159
Epoch [2/5], Loss: 0.0113
Epoch [3/5], Loss: 0.0071
Epoch [4/5], Loss: 0.0047
Epoch [5/5], Loss: 0.0034
Fine-tuning pm_10 model
Epoch [1/5], Loss: 0.0045
Epoch [2/5], Loss: 0.0030
Epoch [3/5], Loss: 0.0028
Epoch [4/5], Loss: 0.0026
Epoch [5/5], Loss: 0.0029
Fine-tuning pm_25 model
Epoch [1/5], Loss: 0.0015
Epoch [2/5], Loss: 0.0012
Epoch [3/5], Loss: 0.0010
Epoch [4/5], Loss: 0.0008
Epoch [5/5], Loss: 0.0008
Fine-tuning sulphur_dioxide model
Epoch [1/5], Loss: 0.0